In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
def compute_loss(y, tx, w, type="mse"):
    """Calculate the loss using mse."""
    N = y.shape[0]
    fw = np.matmul(tx,w)
    if type=="mse":
        return (1/(2*N)*np.matmul(y-fw,(y-fw).T))
    if type=="mae":
        return (1/N*np.sum(np.abs(y-fw)))

def compute_gradient(y, tx, w):
    """Compute the gradient."""
    N=y.size
    e=y-np.matmul(tx,w)
    return(-1/N*np.matmul(tx.T,e))

def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    N=y.size
    e=y-np.matmul(tx,w)
    return(-1/N*np.matmul(tx.T,e))

In [8]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    #linear regression using gradient descent
    w = initial_w
    w = initial_w
    for n_iter in range(max_iters):
        gr=compute_gradient(y,tx,w)
        loss=(compute_loss(y,tx,w))
        w=w-gamma*gr
        #print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return (w, loss)

In [10]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


In [13]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    #linear regression using gradient descent
    batch_size = 1
    w = initial_w
    for n_iter in range(max_iters):
        sum=0
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            sum+=compute_stoch_gradient(minibatch_y,minibatch_tx,w)
        gr=1/minibatch_y.shape[0]*sum
        loss=compute_loss(y,tx, w)
        w=w-gamma*gr
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return (w, loss)